In [1]:
from ipywidgets import FileUpload, Output
import ipywidgets as widgets
from IPython.display import display, Javascript,  HTML
import sys
from io import StringIO
import pandas as pd
import plotly.express as px
import numpy as np 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.signal import find_peaks
from IPython.display import clear_output
import os 
import voila
from collections import namedtuple


In [2]:
# VARIABLES AND WIDGETS *************************************************************************************************

cell_names = ['Left Top','Middle Top','Right Top','Left Bottom','Middle Bottom','Right Bottom']
colors = ['blue','green','red','cyan','magenta','yellow']

    # Mode selector (OPV or OPD)

OPV_or_OPD = widgets.RadioButtons(
            options=['OPV', 'OPD (dark only)', 'OPD (dark and light)'],
            layout={'width': 'max-content'}, # If the items' names are long
            description='Mode :',
            disabled=False
        )

    # Mask format selector 
    
maskFormat = widgets.Dropdown(
            options=['6* 0.16 cm²', '4* 0.09 + 2* 0.25 cm²', '3* 0.27 cm²','single'],
            value= '6* 0.16 cm²',
            description='Mask format :',
            disabled=False,
            layout={'width': 'max-content'}
        )

    # Plot unit selector (mA or A)

mA_or_A = widgets.RadioButtons(
            options=['mA', 'A'],
            layout={'width': 'max-content'}, 
            description='Current Unit :',
            disabled=False  
        )

    # Plot tangent

plot = widgets.RadioButtons(
            options=['Yes', 'No'],
            layout={'width': 'max-content'}, 
            description='Plot tangent lines:',
            style=dict(description_width='initial'),
            disabled=False  
        )
    # Plot scale selector (linear or log)

lin_or_log = widgets.RadioButtons(
            options=['lin', 'log'],
            layout={'width': 'max-content'}, 
            description='Y scale : ',
            disabled=False  
        )

    # OK and Plot buttons

OK_button = widgets.Button(
            description='OK',
            disabled=False,
            button_style='', 
            tooltip='Click me',
            icon='check' 
        )

Plot_button = widgets.Button(                     
            description='Plot', 
            disabled=False,
            button_style='',
            tooltip='Click me',
            icon='line-chart' 
        ) 

surfaceInput = widgets.BoundedFloatText(
    value=0.16,
    min=0,
    max=10.0,
    step=0.1,
    description="Surface for single mask format",
    disabled=False,
    style=dict(description_width='initial')
)


In [3]:
out = Output()
upload=FileUpload(accept='.txt', multiple=True)
upload

FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

In [4]:
display(OPV_or_OPD, widgets.Output())
display(maskFormat, widgets.Output())
display(surfaceInput)

RadioButtons(description='Mode :', layout=Layout(width='max-content'), options=('OPV', 'OPD (dark only)', 'OPD…

Output()

Dropdown(description='Mask format :', layout=Layout(width='max-content'), options=('6* 0.16 cm²', '4* 0.09 + 2…

Output()

BoundedFloatText(value=0.16, description='Surface for single mask format', max=10.0, step=0.1, style=Descripti…

In [5]:
Settings = namedtuple("Settings", ["mode", "mask", "surface"])
Options = namedtuple("Options",["mA_or_A","lin_or_log","plot"])

In [23]:
def NumberOfSamplesAndCells(files, settings):
        
        number_of_samples = int(len(files))
        cell_surfaces = [settings.surface]*number_of_samples
        
        if number_of_samples == 0: 
            print("Upload at least one file.")
            return number_of_samples, cell_surfaces
        
        if settings.mode != 'OPD (dark and light)':

            if settings.mask == '6* 0.16 cm²':
                number_of_samples = int(number_of_samples/6) 
                cell_surfaces=[0.16,0.16,0.16,0.16,0.16,0.16]

            elif settings.mask == '4* 0.09 + 2* 0.25 cm²':
                number_of_samples = int(number_of_samples/6) 
                cell_surfaces=[0.09,0.25,0.09,0.09,0.25,0.09]

            elif maskFormat.value == '3* 0.27 cm²':
                number_of_samples = int(number_of_samples/3) 
                cell_surfaces=[0.27,0.27,0.27]


        else:
            if settings.mask == '6* 0.16 cm²':
                number_of_samples = int(number_of_samples/12) 
                cell_surfaces=[0.16,0.16,0.16,0.16,0.16,0.16]

            elif settings.mask == '4* 0.09 + 2* 0.25 cm²':
                number_of_samples = int(number_of_samples/12) 
                cell_surfaces=[0.09,0.25,0.09,0.09,0.25,0.09]
            
            elif settings.mask == '3* 0.27 cm²':
                number_of_samples = int(number_of_samples/6) 
                cell_surfaces=[0.27,0.27,0.27]
            
        return number_of_samples, cell_surfaces
    
def read_file(content,surface,options):
    s = "" 
    for line in content.split(b'\n'):
        if "Source;Delais(s);Mesure ;Temps(s);Mesure/Source;Source/Mesure;Puissance" in line.decode('latin-1'):
            s = ""
        s +=line.decode('latin-1')

    df = pd.read_csv(StringIO(s), sep=";")
    
    if options.mA_or_A == 'A':
        df['J'] = df['Mesure ']/surface
        df['I'] = df['Mesure ']
    else:
        df['J'] = df['Mesure ']*1000/surface
        df['I'] = df['Mesure ']*1000

    df['Power'] = df['J']*df['Source']
    df['Jab'] = df['J'].abs()
    
    return df

def sort_files(uploads, nb_cells, settings):
    darks = []
    lights =[]

    f = [uploads[i:i+nb_cells] for i in range(0, len(uploads), nb_cells)]

    if settings.mode == 'OPD (dark and light)':
        darks = f[::2] #even
        lights = f[1::2] #odd
    
    return f, darks, lights

In [24]:
def extract(df,i,inc):
    start = i - inc if i - inc > 0 else 0
    end = i + inc if i+inc < len(df) else len(df)
    a = df.iloc[start:end]
    
    xData = a.Source
    yData = a.J
    return xData, yData

def tangente(xData,yData,x0,options):
    
    fittedParameters = np.polyfit(xData, yData, 5)
    
    xModel = np.linspace(min(xData), max(xData))
    yModel = np.polyval(fittedParameters, xModel)
    data = {'x':xModel, 'y':yModel}
    dffitted = pd.DataFrame(data)
    
     # polynomial derivative from numpy
    deriv = np.polyder(fittedParameters)

    # for plotting
    minX = min(xData)
    maxX = max(xData)

    pointVal = x0
    y_value_at_point = np.polyval(fittedParameters, pointVal)
    slope_at_point = np.polyval(deriv, pointVal)

    ylow = (minX - pointVal) * slope_at_point + y_value_at_point
    yhigh = (maxX - pointVal) * slope_at_point + y_value_at_point
    
    data = {'x':xModel, 'y':yModel}
    dffitted = pd.DataFrame(data)
    
    R = 1/slope_at_point if options.mA_or_A == 'A' else 1000/slope_at_point
    return R, [minX,maxX], [ylow, yhigh]

#Tangentes 
def ExtractRshunt(df,fig,options):
    xData, yData = extract(df,df[df.J > np.interp(0, df.Source, df.J)].index[0],50)
    Rshunt, linex, liney = tangente(xData,yData,0,options)
    if options.plot: 
        fig.add_trace(go.Scatter(x=linex,y=liney,mode='lines',  line=dict(color="#808080", dash="dash"),showlegend=False), row =1, col = 1)
    return Rshunt

def ExtractRserie(df,fig,options):
    xData, yData = extract(df,df[df.J > 0].index[0],10)
    Voc = round(np.interp(0, df.J, df.Source),2)
    Rserie, linex, liney = tangente(xData,yData,Voc,options)
    if options.plot:
        fig.add_trace(go.Scatter(x=linex,y=liney,mode='lines', line=dict(color="#808080", dash="dash"),showlegend=False), row =1, col = 1)
    return Rserie

In [25]:
def add_lines(df,fig,color,title,dark):
    if dark:
        fig1 = px.line(df,x="Source", y="Jab").update_traces(line_color=color, name = title, showlegend=True)
    else: 
        fig1 = px.line(df,x="Source", y="J").update_traces(line_color=color, name = title, showlegend=True)
        
    fig2 = px.line(df,x="Source", y="Power").update_traces(line_color=color, name = title, showlegend=False)
    fig3 = px.line(df,x="Source", y="Mesure ").update_traces(line_color=color, name = title, showlegend=False)
    
    fig.add_trace(fig1['data'][0], row=1, col=1)
    fig.add_trace(fig2['data'][0], row=1, col=2)
    fig.add_trace(fig3['data'][0], row=1, col=3)
    
def UpdateAxisProperties(fig,options):
    fig.update_layout(
        showlegend=False,
        autosize=False,
        width=2000,
        height=700,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=3
        )
    )
    # Update xaxis properties
    fig.update_xaxes(title_text='Voltage (V)', row=1, col=1)
    fig.update_xaxes(title_text='Voltage (V)', row=1, col=2)
    fig.update_xaxes(title_text='Voltage (V)', row=1, col=3)

    # Update yaxis properties 
    if options.mA_or_A == 'A':
        fig.update_yaxes(title_text='J (A/cm\u00b2)', row=1, col=1)       
        fig.update_yaxes(title_text='P (W/cm\u00b2)', row=1, col=2)
        fig.update_yaxes(title_text='I (A)', row=1, col=3)
    else:
        fig.update_yaxes(title_text='J (mA/cm\u00b2)', row=1, col=1)       
        fig.update_yaxes(title_text='P (mW/cm\u00b2)', row=1, col=2)
        fig.update_yaxes(title_text='I (mA)', row=1, col=3)

    if options.lin_or_log == 'log':
        fig.update_yaxes(type="log")

    fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')
    fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black')

In [19]:
def add_data(df, data, fig, options, sample_name, cell_name, surface, dark):
    
    if not dark:
        Jsc = round(np.abs(np.interp(0, df.Source, df.J)),2)
        Voc = round(np.interp(0, df.J, df.Source),2)
        Jmax = round(np.abs(np.array(df[df.Power == min(df.Power)].J)[0]),2)
        Vmax = round(np.array(df[df.Power == min(df.Power)].Source)[0],2);
        FF = round(((Vmax*Jmax)/(Voc*Jsc))*100,2)
        PCE = round((Voc*Jsc*FF)/100,2)

        Rserie = ExtractRserie(df, fig, options)
        Rshunt = ExtractRshunt(df, fig, options)
        
        df.to_excel(sample_name +' - light plots - ' + cell_name + '.xlsx')
        data.append([cell_name,Voc,Jsc,Vmax,Jmax,round(Voc*Jsc,2),round(Vmax*Jmax,2),FF,PCE,surface,Rshunt,Rserie])    #CARC

    else: 
        # Calculating FoM
        Jmin = min(df.Jab)
        V_Jmin = np.interp(Jmin, df.J, df.Source)
        J_0 = np.abs(np.interp(0, df.Source, df.J))
        J_1 = np.abs(np.interp(-1, df.Source, df.J))
        J_2 = np.abs(np.interp(-2, df.Source, df.J))
        minPower = min(df.Power)
        
        df.to_excel(sample_name +' - dark plots - ' + cell_name + '.xlsx')
        data.append([cell_name,Jmin,V_Jmin, J_0, J_1, J_2, surface])  
    
def create_dftable(data, options, dark):
    if dark: 
        dftable = pd.DataFrame(data, columns=['Cell', 'Jmin ('+options.mA_or_A+'/cm²)','V(Jmin)(V) ','J(0)','J(-1)','J(-2)','surface (cm²)'])         
        label = ' - dark FoM'
        
    else:
        label = ' - light FoM'
        dftable = pd.DataFrame(data, columns=['Cell', 'Voc (V)','J ('+options.mA_or_A+'/cm²)','Vmax','Jmax',
                                              'Voc*Jsc','Vmax*Jmax','FF(%)','PCE (%)','surface',
                                              'Rshunt (Ω/cm²)', 'Rserie (Ω/cm²)'])        
    return label, dftable

def main_loop(files, splitted_files, cell_surfaces,sample_names,settings,options, dark):
    data = []
    if dark: 
        print('\n DARK RESULTS : \n')
    else: 
        print('\n LIGHT RESULTS : \n')
        

    for sample in splitted_files:
        fig = make_subplots(rows=1, cols=3)
        sample_index = splitted_files.index(sample)
        
        for cell in sample:
            cell_index = sample.index(cell)
            content = files[cell]['content']
            surface = cell_surfaces[cell_index]
            df = read_file(content,surface,options)
            cell_name = cell_names[cell_index]
            sample_name = sample_names[sample_index]
            
            add_lines(df,fig,colors[cell_index%len(colors)],cell_name,dark)
            UpdateAxisProperties(fig,options)
            fig.update_layout(title_text=sample_name)  
            add_data(df, data, fig, options, sample_name, cell_name, surface, dark)
        
        label, dftable = create_dftable(data, options, dark) 
        print("Sample : ", sample_name)          
        display(dftable)
        dftable.to_excel(sample_name+label+ '.xlsx')
        data = []
        fig.update_layout(showlegend=True)
        fig.show() 

In [21]:
def OK_button_clicked(b):
    
    with out:
        out.clear_output()
        sample_names=[]
        
        files  = upload.value
        files_name = list(files)
        
        settings = Settings(mode=OPV_or_OPD.value, mask = maskFormat.value, surface = surfaceInput.value)
        number_of_samples, cell_surfaces = NumberOfSamplesAndCells(files, settings)
        if number_of_samples < 1 : return
        print(settings.mode, ' mode. ', str(number_of_samples)+' sample(s).')   
        number_of_cells = len(cell_surfaces) if settings.mask != 'single' else 1
        
        splitted_files, darks, lights = sort_files(files_name, number_of_cells, settings)
        
    # SAMPLE NAMES INPUT ****************************************************************************************************

        name_widgets = []

        for i in range(number_of_samples):
            name_widgets.append(widgets.Text(          
               value='',
               placeholder='Write your sample name',
               description="Sample °" + str(i+1),
               disabled=False
             ))

            display(name_widgets[i], widgets.Output())    

        display(plot)
    # Some options ***********************************************************************************************
        if settings.mode != 'OPV':
            display(lin_or_log, widgets.Output())
            display(mA_or_A, widgets.Output())
        
    # plot button function ***************************************************************************************
        def plot_button_clicked(b):
            with out:
                out.clear_output()
                if settings.mode != 'OPV':
                    options = Options(mA_or_A = mA_or_A.value, lin_or_log = lin_or_log.value, plot = plot.value == 'Yes')
                else: 
                    options = Options(mA_or_A = 'mA', lin_or_log = 'lin', plot = plot.value == 'Yes')
            
                for i in name_widgets:
                    if i.value !="":
                        sample_names.append(i.value)  
                    else: 
                        sample_names.append("Sample"+str(name_widgets.index(i)))
                
                if settings.mode == "OPV":
                    main_loop(files,splitted_files, cell_surfaces,sample_names,settings,options,False)
                elif settings.mode == "OPD (dark only)":
                    main_loop(files,splitted_files, cell_surfaces,sample_names,settings,options,True)
                else: 
                    main_loop(files, darks, cell_surfaces,sample_names,settings,options,True)
                    main_loop(files, lights, cell_surfaces,sample_names,settings,options,False)
                    
                
# PLOT BUTTON EXECUTION ***************************************************************************************
        
        display(Plot_button, widgets.Output())
        Plot_button.on_click(plot_button_clicked) 
        
    out

In [22]:
# OK BUTTON EXECUTION ******************************************************************************************

OK_button.on_click(OK_button_clicked)
display(OK_button, widgets.Output())

out

Button(description='OK', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()

Output()